In [5]:
import sys
import os

In [6]:
sys.path.append('\\'.join(os.getcwd().split('\\')[:-2]))

In [7]:
import pandas as pd
import numpy as np
import pickle

from amt import TurkResults2Label, LabelGetter

In [8]:
batch1 = pd.read_csv('C:/users/tom work/documents/amt/Batch_2431673_batch_results.csv').drop_duplicates(subset='_id')
batch2 = pd.read_csv('C:/users/tom work/documents/amt/Batch_2475097_batch_results.csv').drop_duplicates(subset='_id')
batch3 = pd.read_csv('C:/users/tom work/documents/amt/Batch_2475220_batch_results.csv').drop_duplicates(subset='_id')

In [9]:
df = pd.concat([batch1, batch2, batch3])

In [10]:
df.shape

(3997, 8)

In [11]:
set(df.Answer.values)

{'Alcohol Related::Discussion',
 'Alcohol Related::Promotional Content',
 'First Person - Alcohol::Casual Drinking',
 'First Person - Alcohol::Heavy Drinking',
 'First Person - Alcohol::Looking to drink',
 'First Person - Alcohol::Reflecting on drinking',
 'Not Alcohol Related',
 '{}'}

In [12]:
df= df[df.Answer != '{}']

In [13]:
df['labels'] = df.Answer[df.Answer != '{}'].apply(TurkResults2Label.parse_to_labels)

In [14]:
df.head()

,Answer,Date,HITID,Worker,_id,_id.1,random_number,text,labels
0,Alcohol Related::Discussion,2016-06-14 20:46:42 UTC,3D4BBDG7ZHGFFKR3R4H5X9KLV8AC35,A3MZ9RQJX9T656,642898530122620928,NaN,-9201186384286387794,"b'""I wasn\'t a smoker till I smelt the cigaret...","{'first_person': 0, 'alcohol_related': 0, 'alc..."
1,Alcohol Related::Discussion,2016-06-14 20:46:34 UTC,3RIHDBQ1NEI7IEATJE0L4ZQ78RWHMZ,A2QLSHXNCHBRN4,643649753327124480,NaN,-9201020976683528985,b'@jigglingbreasts yall must be fun and have a...,"{'first_person': 0, 'alcohol_related': 0, 'alc..."
2,First Person - Alcohol::Looking to drink,2016-06-14 20:46:24 UTC,33Q5P9PUSP6G0DYWCBCE1AP0A38ZCO,A18TCR555RWUZV,645646883252236289,NaN,-9201279431476675312,b'I feel like shit drunk too much',"{'first_person': 1, 'first_person_level': 1, '..."
3,Not Alcohol Related,2016-06-14 20:46:55 UTC,3YD0MU1NC2L9NOVS92O44V6D07AA75,A3D9L5CQXK1YJL,641523980826902528,NaN,-9222539614720812479,"b""when you don't have shit to drink so you eat...",{'alcohol': 0}
4,Not Alcohol Related,2016-06-14 20:46:45 UTC,32LAQ1JNT99G0V6XSA8W9SAPW7TTUC,A3D9L5CQXK1YJL,649030562301247488,NaN,-9198559142838941823,b'@Jeep_Jones2 is butt hurt now bc and am stud...,{'alcohol': 0}


In [15]:
L = LabelGetter(df)

In [17]:
X, y = L.get_alcohol()

In [18]:
from pipelines.simple import make_classifier

In [19]:
clf_alc = make_classifier()
clf_fpa = make_classifier()
clf_fpl = make_classifier()

In [23]:
from sklearn.cross_validation import train_test_split

In [24]:
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=23)

In [25]:
clf_alc.fit(X_train, y_train)

Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=200000, min_df=1,
        ngram_range=(1, 3), norm='l2',...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [27]:
from classification.reporting import ClassificationReporting

In [28]:
reporting = ClassificationReporting(clf_alc, X_train, X_test, y_train, y_test, 2)

In [29]:
report = reporting.set_name("Test Classifier").set_level('alc').create_report(1)

C:\Users\Tom Work\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:976: DeprecationWarning: From version 0.18, binary input will not be handled specially when using averaged precision/recall/F-score. Please use average='binary' to report only the positive class performance.
  'positive class performance.', DeprecationWarning)
C:\Users\Tom Work\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:976: DeprecationWarning: From version 0.18, binary input will not be handled specially when using averaged precision/recall/F-score. Please use average='binary' to report only the positive class performance.
  'positive class performance.', DeprecationWarning)


Training Results
~~~~~~~~~~~~~~~~
confusion_matrix
[[1018, 2], [0, 1935]]


classification_report
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      1020
          1       1.00      1.00      1.00      1935

avg / total       1.00      1.00      1.00      2955



f1_score
0.9994834710743802


accuracy_score
0.9993231810490694




Testing Results Results
~~~~~~~~~~~~~~~~~~~~~~~
confusion_matrix
[[194, 146], [57, 588]]


classification_report
             precision    recall  f1-score   support

          0       0.77      0.57      0.66       340
          1       0.80      0.91      0.85       645

avg / total       0.79      0.79      0.79       985



f1_score
0.8527918781725887


accuracy_score
0.7939086294416243






In [30]:
X, y = L.get_first_person()
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=24)
clf_fpa.fit(X_train, y_train)
reporting = ClassificationReporting(clf_fpa, X_train, X_test, y_train, y_test, 2)
report = reporting.set_name("Test Classifier").set_level('fpa').create_report(1)

C:\Users\Tom Work\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:976: DeprecationWarning: From version 0.18, binary input will not be handled specially when using averaged precision/recall/F-score. Please use average='binary' to report only the positive class performance.
  'positive class performance.', DeprecationWarning)
C:\Users\Tom Work\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:976: DeprecationWarning: From version 0.18, binary input will not be handled specially when using averaged precision/recall/F-score. Please use average='binary' to report only the positive class performance.
  'positive class performance.', DeprecationWarning)


Training Results
~~~~~~~~~~~~~~~~
confusion_matrix
[[829, 3], [0, 1103]]


classification_report
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       832
          1       1.00      1.00      1.00      1103

avg / total       1.00      1.00      1.00      1935



f1_score
0.9986419194205525


accuracy_score
0.9984496124031008




Testing Results Results
~~~~~~~~~~~~~~~~~~~~~~~
confusion_matrix
[[128, 142], [91, 284]]


classification_report
             precision    recall  f1-score   support

          0       0.58      0.47      0.52       270
          1       0.67      0.76      0.71       375

avg / total       0.63      0.64      0.63       645



f1_score
0.7091136079900126


accuracy_score
0.6387596899224807






In [31]:
X, y = L.get_first_person_label()
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.33, random_state=25)
clf_fpl.fit(X_train, y_train)
reporting = ClassificationReporting(clf_fpl, X_train, X_test, y_train, y_test, 3)
report = reporting.set_name("Test Classifier").set_level('fpl').create_report(1)

Training Results
~~~~~~~~~~~~~~~~
confusion_matrix
[[417, 0, 0], [0, 342, 0], [0, 0, 231]]


classification_report
             precision    recall  f1-score   support

          0       1.00      1.00      1.00       417
          1       1.00      1.00      1.00       342
          2       1.00      1.00      1.00       231

avg / total       1.00      1.00      1.00       990



f1_score
1.0


accuracy_score
1.0




Testing Results Results
~~~~~~~~~~~~~~~~~~~~~~~
confusion_matrix
[[111, 50, 35], [54, 102, 18], [46, 33, 39]]


classification_report
             precision    recall  f1-score   support

          0       0.53      0.57      0.55       196
          1       0.55      0.59      0.57       174
          2       0.42      0.33      0.37       118

avg / total       0.51      0.52      0.51       488



f1_score
0.5115006437464084


accuracy_score
0.5163934426229508






In [176]:
def extract_terms(clf):
    logreg = clf.steps[2][1]
    tfidf = clf.steps[1][1]
    features = tfidf.get_feature_names()
    norm = np.linalg.norm(logreg.coef_[0])
    terms = dict()
    
    term_coef = [(logreg.coef_[0][i]/norm , features[i]) for i in range(len(features))]
    term_coef.sort(key = lambda x: x[0])
    for i in range(len(features)):
        terms[term_coef[i][1]] =i/len(term_coef)
    print(term_coef[-10:])
    print(term_coef[:10])
    return terms

In [103]:
sept_alc = extract_terms(clf_alc)

[(0.0321598758718653, 'tequila'), (0.033560632615295884, 'whiskey'), (0.033980068043929085, 'vodka'), (0.036696961502476949, 'tonight'), (0.038397893117968193, 'liquor'), (0.040381943841632052, 'beers'), (0.061345768360699007, 'alcohol'), (0.06812575205401912, 'wine'), (0.10374141499591916, 'beer'), (0.11341271623148504, 'drunk')]
[(-0.1546973458487052, 'shot'), (-0.097028541556362424, 'water'), (-0.074889984715651003, 'coffee'), (-0.050488717913707494, 'wasted'), (-0.040220747286077656, 'bar'), (-0.037556531552851644, 'tea'), (-0.035608297855315357, 'shots'), (-0.032248363172284834, 'there'), (-0.03158807238996602, 'starbucks'), (-0.03144706261432275, 'shot at')]


In [47]:
len(tfidf.get_feature_names()

81872

In [67]:
clf_june = pickle.load(open('C:/Users/Tom Work/PycharmProjects/nyu-twipsy/simple classifiers/clf_alc_simple.p', 'rb'))

In [72]:
len(clf_june.steps[1][1].get_feature_names())

200000

In [109]:
june_alc = extract_terms(clf_june)

[(0.054879980616541611, 'vodka'), (0.058028853637784539, 'liquor'), (0.062373414477272189, 'beers'), (0.062887475188977984, 'alcohol'), (0.066689534473448817, 'drinks'), (0.078043593748398699, 'drink'), (0.087151249914254436, 'wine'), (0.10224321931602527, 'drinking'), (0.1210161204938294, 'beer'), (0.15762680669910298, 'drunk')]
[(-0.044513583764149961, 'coffee'), (-0.04337891566198418, '✔️'), (-0.040249862512635855, 'female'), (-0.03538322429403426, 'starbucks'), (-0.034744132241989002, 'tea'), (-0.031203324259672427, 'females'), (-0.031068073840744743, 'water'), (-0.030138678806763096, 'alex'), (-0.028354788977550029, 'public'), (-0.02808251657756753, 'root')]


In [110]:
june_alc['root beer']

6.5e-05

In [111]:
combined = set(june_alc.keys()).intersection(set(sept_alc.keys()))

In [113]:
len(combined)

15266

In [118]:
percentile_diff = [(a, june_alc[a]-sept_alc[a]) for a in combined]
    

In [120]:
percentile_diff.sort(key=lambda x: abs(x[1]))

In [121]:
percentile_diff[:10]

[('know the', 1.7686144224704492e-06),
 ('drunk', 7.214188000759236e-06),
 ('of a bottle', -1.0077193668078799e-05),
 ('take', -1.2017783857731779e-05),
 ('5', 1.311315223762044e-05),
 ('alcohol', 1.3856752003160722e-05),
 ('beer', 1.4428376001629495e-05),
 ('wine', 1.6642564002355975e-05),
 ('water', 1.778581199921829e-05),
 ('beers', 2.1070940003919958e-05)]

In [124]:
sept_alc['drank']

0.0024061950361539963

In [130]:
sept_alc['september']

0.9852941176470589

In [125]:
percentile_diff[-100:]

[('ima', 0.9813447478991597),
 ('break', 0.9814260396716826),
 (', that', -0.9821200547195622),
 ('dinner &amp;', 0.9822770656634746),
 ('$10', 0.9823588078952512),
 ('if the', -0.9823864559312098),
 ('but my', 0.9826565663474692),
 ("spanish wine '", -0.9829902745749463),
 ('at a', 0.9830083603674028),
 ('valley', 0.9832566005471955),
 ('green', 0.9834037394957983),
 ('to sleep', -0.9834245798319328),
 ('played', -0.9834914012116475),
 ('with me', 0.9835185391831152),
 ('haha', -0.9837808608559704),
 ("it 's", 0.9841403263631033),
 ('sweet', -0.9844452990033222),
 ('like its', 0.9847166630838382),
 ('so happy', 0.9847593521594684),
 ('view', -0.9854465839359),
 ('yesterday', 0.9856903781512606),
 ('in high', -0.9858276646472542),
 ('in high school', -0.985834878835255),
 ('feelin', 0.9862802960719171),
 ('was', -0.9863778131717803),
 ('like drinking', -0.9867822307992964),
 ("i'm ready", -0.9868831502833691),
 ('just bought', 0.9873899550517882),
 ('much', 0.9873932294313074),
 ('the 

In [132]:
os.getcwd()

'C:\\Users\\Tom Work\\PycharmProjects\\twipsy-lite\\notebooks\\python'

In [172]:
clf_alc_char = pickle.load(open('C:\\Users\\Tom Work\\PycharmProjects\\nyu-twipsy/pickles/clf_alc_UPDATED.p', 'rb'))

In [188]:
feats = clf_alc_char.steps[0][1].transformer_list[0][1].steps[1][1].get_feature_names()

In [197]:
len(feats)

71233

In [198]:
len(clf_alc_char.steps[2][1].coef_[0])

71233

In [199]:
combined = [(clf_alc_char.steps[2][1].coef_[0][i], feats[i]) for i in range(len(feats))]

In [200]:
combined.sort(key = lambda x: x[0])

In [201]:
combined[:10]

[(-32.332502549897214, 'ale'),
 (-21.877597885799165, 'gin'),
 (-14.216205262531048, 'le'),
 (-12.711171431912025, 'al'),
 (-12.518261502841082, 'gi'),
 (-11.527543050493119, ' wate'),
 (-11.483684228292102, ' ✔'),
 (-11.319546157805012, 'n lov'),
 (-11.042636216281647, 'mt'),
 (-10.595989916665161, 'd fuc')]

In [202]:
combined[-10:]

[(10.636194652462736, ' win'),
 (11.654810418080649, 'bee'),
 (12.19898358728522, 'eer'),
 (14.898349001173823, ' bee'),
 (15.068612111562615, ' drun'),
 (15.115428548800434, ' dru'),
 (15.270350606613764, 'wine'),
 (15.813868941675237, 'beer'),
 (17.214012928023035, ' beer'),
 (18.091174838254151, ' wine')]

In [133]:
pickle.dump(clf_alc, open('C:\\Users\\Tom Work\\PycharmProjects\\twipsy-lite/clf_alc_sept.p', 'wb'))

In [134]:
pickle.dump(clf_fpa, open('C:\\Users\\Tom Work\\PycharmProjects\\twipsy-lite/clf_fpa_sept.p', 'wb'))

In [137]:
df_sept = pd.read_csv('E:/from-hpc/sept-trained-summary/tweets_1predict.csv')

In [229]:
df_sample = pd.read_csv('E:/Sept-March 2015/csv with offset/june-small-block/tweets_0.csv')

In [236]:
df_sample.sample(10)

,Unnamed: 0,id,created_at,utc_offset,text
27389,5429,6.058118e+17,Tue Jun 02 19:02:52 +0000 2015,-21600.0,@chrisjcrespinn bangers.😍
44738,22774,6.058142e+17,Tue Jun 02 19:12:15 +0000 2015,NaN,Thank you Jesus!!
50455,28491,6.058150e+17,Tue Jun 02 19:15:20 +0000 2015,-14400.0,Tony Abbott crushes attempt to legalise gay ma...
12104,12104,6.058098e+17,Tue Jun 02 18:54:41 +0000 2015,NaN,@bootyall21 yayyy😻 I don't even know how I'm g...
27152,5192,6.058118e+17,Tue Jun 02 19:02:45 +0000 2015,-18000.0,@donrober_ #Moterrey #México Checa el texto d...
35980,14020,6.058130e+17,Tue Jun 02 19:07:34 +0000 2015,-25200.0,Red Robin &gt;
54303,32339,6.058155e+17,Tue Jun 02 19:17:25 +0000 2015,-14400.0,It was so awkward.
70599,48631,6.058178e+17,Tue Jun 02 19:26:23 +0000 2015,-18000.0,Entourage movie tonight!!! @jeoffryreynoldz @Q...
71200,49232,6.058178e+17,Tue Jun 02 19:26:42 +0000 2015,-10800.0,I know im gonna cry tonight bc pookie 😢 #bitte...
44184,22220,6.058141e+17,Tue Jun 02 19:11:58 +0000 2015,-18000.0,@KariKouture u can't answer your phone ? I bee...


In [216]:
df_other = pd.read_csv('E:/from-hpc/sept-trained-summary/tweets_2predict.csv')

In [151]:
df_other.shape

(67195, 15)

In [225]:
import re

In [ ]:
def remove_trash(text):
    

In [227]:
df_other[df_other.predict_alc > 0.75].sample(10)

,Unnamed: 0,Unnamed: 0.1,id,created_at,utc_offset,text,predict_alc,predict_fpa,predict_fpa|alc,predict_present|fpa,predict_future|fpa,predict_past|fpa,predict_present,predict_future,predict_past
35460,53170,55795,7.080419e+17,Thu Mar 10 21:28:27 +0000 2016,-25200.0,"2 years ago on my birthday, I got a concussion...",0.880527,0.660463,0.750077,0.391390,0.010237,0.598373,0.258499,0.006761,0.395203
63503,94483,97107,7.080472e+17,Thu Mar 10 21:49:43 +0000 2016,-28800.0,First outdoor meet was a success! ????????????...,0.888995,0.433050,0.487124,0.723931,0.000308,0.275761,0.313499,0.000133,0.119418
10544,16083,18713,7.080372e+17,Thu Mar 10 21:09:45 +0000 2016,-18000.0,"La Tech 23, ODU 23, 3:10. After last game, thi...",0.946711,0.378968,0.400300,0.085235,0.075969,0.838796,0.032301,0.028790,0.317877
49030,73214,75838,7.080444e+17,Thu Mar 10 21:38:36 +0000 2016,-28800.0,Lmao I just realized I told this boy I was don...,0.833837,0.770071,0.923527,0.020768,0.009512,0.969720,0.015993,0.007325,0.746753
58582,87206,89830,7.080463e+17,Thu Mar 10 21:46:11 +0000 2016,-18000.0,@NYGovCuomo but not Long Island,0.959958,0.684345,0.712890,0.894855,0.002799,0.102346,0.612389,0.001915,0.070040
27872,41971,44596,7.080404e+17,Thu Mar 10 21:22:40 +0000 2016,-21600.0,???????????? https://t.co/ZK5y2gdjYS,0.785222,0.354372,0.451301,0.999515,0.000290,0.000195,0.354200,0.000103,0.000069
65740,97850,100474,7.080476e+17,Thu Mar 10 21:51:16 +0000 2016,-21600.0,you must need a babysitter or some? $25 a hour...,0.892841,0.381219,0.426973,0.190995,0.807452,0.001553,0.072811,0.307816,0.000592
46926,70054,72678,7.080440e+17,Thu Mar 10 21:36:54 +0000 2016,-18000.0,I miss bitch u guessed it memes.,0.805637,0.450156,0.558758,0.350592,0.198844,0.450563,0.157821,0.089511,0.202824
35950,53903,56527,7.080419e+17,Thu Mar 10 21:28:39 +0000 2016,-18000.0,"@adriangtz95 ya est??, ahorita remontada del Utd",0.856225,0.562717,0.657208,0.769258,0.201924,0.028818,0.432875,0.113626,0.016216
50644,75583,78207,7.080447e+17,Thu Mar 10 21:39:46 +0000 2016,-18000.0,Arnav dhingo_warrior doin' work with the rest ...,0.917238,0.474188,0.516973,0.918984,0.059159,0.021857,0.435771,0.028052,0.010365


In [219]:
df_june.head()

,Unnamed: 0,Unnamed: 0.1,id,created_at,utc_offset,text,predict_alc,predict_fpa,predict_fpa|alc,predict_present|fpa,predict_future|fpa,predict_past|fpa,predict_present,predict_future,predict_past
0,0,0,6.058081e+17,Tue Jun 02 18:48:04 +0000 2015,-25200.0,"Miquan ass just said ""A couple Smarticle Parti...",0.703745,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,1,6.058081e+17,Tue Jun 02 18:48:04 +0000 2015,-25200.0,hey... Have a great day ????,0.368690,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,6,6,6.058081e+17,Tue Jun 02 18:48:04 +0000 2015,-36000.0,Lmfao story of my life! @ The Pandalee Playhou...,0.901191,0.594294,0.659455,0.872571,0.002511,0.124918,0.518564,0.001492,0.074238
3,7,7,6.058081e+17,Tue Jun 02 18:48:04 +0000 2015,-14400.0,@vancemeek my bad man got busy for a few days ...,0.746622,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,8,8,6.058081e+17,Tue Jun 02 18:48:04 +0000 2015,-14400.0,Don't lose sight of your dreams and ambitions.,0.295956,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [217]:
df_other[df_other.predict_fpa > 0.85].sample(10).text

10549                        I am cracking up ????????????
10087    Maybe one day I'll be able to not get strep wh...
28180    I fell asleep during 6th hour and am just now ...
33198    My #otterbox has saved the day.....while #wash...
11071    I thought about making that awkward but I kept...
64788        I'm running a slight fever, but I'm here! Lol
7914     Bout to go swimming when I finish eating . ???...
27792    I've been eating so much these last couple mon...
54081                              One thing after another
42582                          Kinda wanna go grab a drink
Name: text, dtype: object

In [162]:
sum(df_other['predict_fpa|alc'] > 0.75)

1393

In [203]:
df_june = pd.read_csv('E:/from-hpc/tweets_0predict.csv')

In [204]:
df_june.shape

(65110, 15)

In [215]:
df_june[df_june.predict_fpa > 0.85].sample(10).text

18884    so crazy to think that if I keep pushing mysel...
64411                       At Koreatown getting Thai food
1785     but I'm more interested in Panera Bread rn bc ...
36919    Aww this is some BULL SH!# ..... I guess rules...
38858                  @XHVENA never a sincere one tho????
28872    i've grown tf up .. cause i would of been disr...
51655    me and @Flatulant_bryan last night http://t.co...
26272    Been working so much I didn't even get to eat ...
43042    I though this concept was @ContraNYC  https://...
62877    @Brendalchuck Pitching wasn't spotless but we ...
Name: text, dtype: object

In [165]:
df_other.head()

,Unnamed: 0,Unnamed: 0.1,id,created_at,utc_offset,text,predict_alc,predict_fpa,predict_fpa|alc,predict_present|fpa,predict_future|fpa,predict_past|fpa,predict_present,predict_future,predict_past
0,0,2631,7.080350e+17,Thu Mar 10 21:01:15 +0000 2016,-18000.0,@ILoveYouMari_ my butt hole was burning!!!,0.018059,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
1,2,2633,7.080350e+17,Thu Mar 10 21:01:19 +0000 2016,-18000.0,@KsKaniac_7 @BINJURED I agree TOTALLY. Was edi...,0.566069,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2,5,2636,7.080350e+17,Thu Mar 10 21:01:19 +0000 2016,-28800.0,Another classic! #CharlieSoul #soul https://t....,0.986030,0.377706,0.383057,0.993774,0.003096,0.00313,0.375354,0.001169,0.001182
3,7,2638,7.080350e+17,Thu Mar 10 21:01:19 +0000 2016,-21600.0,@KIM0SABI jaja... ando en las calles de Laredo!,0.060941,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
4,8,2639,7.080350e+17,Thu Mar 10 21:01:19 +0000 2016,-21600.0,Have we really still not caught Kony?,0.328589,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000


In [164]:
df_sept.shape

(62811, 15)

In [170]:
df_sept[df_sept['predict_fpa'] > 0.95]

,Unnamed: 0,Unnamed: 0.1,id,created_at,utc_offset,text,predict_alc,predict_fpa,predict_fpa|alc,predict_present|fpa,predict_future|fpa,predict_past|fpa,predict_present,predict_future,predict_past
262,409,100409,638571921366470656,Tue Sep 01 04:39:44 +0000 2015,-14400.0,These last few nights have been so sleepless,0.960042,0.957866,0.997734,0.172799,0.297682,0.529519,0.165519,0.285140,0.507208
393,618,100618,638571942799282176,Tue Sep 01 04:39:49 +0000 2015,-25200.0,I need youssssss http://t.co/lxFqTAFyyQ,0.956819,0.956623,0.999796,0.561955,0.438044,0.000002,0.537579,0.419043,0.000002
461,730,100730,638571954287480833,Tue Sep 01 04:39:52 +0000 2015,-25200.0,I need a facing haircut,0.951941,0.950458,0.998442,0.000005,0.999405,0.000590,0.000005,0.949893,0.000560
538,861,100861,638571967533187072,Tue Sep 01 04:39:55 +0000 2015,-10800.0,I legit binge eat when I'm drunk ????,0.967491,0.959811,0.992063,0.518547,0.007492,0.473961,0.497707,0.007191,0.454914
553,888,100888,638571969831571457,Tue Sep 01 04:39:56 +0000 2015,-25200.0,@___saraaaaaaa @ me,0.992313,0.983322,0.990940,0.020763,0.928347,0.050890,0.020417,0.912864,0.050041
581,932,100932,638571975800160256,Tue Sep 01 04:39:57 +0000 2015,-14400.0,So I was just half sleep now I'm just up ????....,0.982870,0.977689,0.994729,0.667937,0.030865,0.301198,0.653034,0.030176,0.294478
638,1028,101028,638571986826821632,Tue Sep 01 04:40:00 +0000 2015,-18000.0,@reignaaa_ i just want a tb,0.963276,0.954245,0.990625,0.006812,0.990865,0.002324,0.006500,0.945528,0.002217
852,1368,101368,638572020389806081,Tue Sep 01 04:40:08 +0000 2015,-14400.0,The only person I wanna speak to rn is oomf,0.974011,0.972219,0.998161,0.041363,0.722042,0.236595,0.040214,0.701983,0.230022
884,1414,101414,638572027243163648,Tue Sep 01 04:40:10 +0000 2015,-10800.0,@wingosvsn i gotchu right now,0.963424,0.954652,0.990895,0.348408,0.646501,0.005091,0.332609,0.617183,0.004860
939,1498,101498,638572036512595968,Tue Sep 01 04:40:12 +0000 2015,-25200.0,Fuck this shit I'm out ???????????????????????...,0.983388,0.972043,0.988463,0.842816,0.008770,0.148414,0.819254,0.008525,0.144264
